In [1]:
!pip3 install -q ktrain

     |████████████████████████████████| 25.2MB 126kB/s 
     |████████████████████████████████| 421.8MB 26kB/s 
     |████████████████████████████████| 6.7MB 44.7MB/s 
     |████████████████████████████████| 983kB 49.0MB/s 
     |████████████████████████████████| 245kB 61.5MB/s 
     |████████████████████████████████| 675kB 50.4MB/s 
     |████████████████████████████████| 471kB 55.1MB/s 
     |████████████████████████████████| 450kB 62.9MB/s 
     |████████████████████████████████| 3.9MB 58.7MB/s 
     |████████████████████████████████| 3.8MB 45.7MB/s 
     |████████████████████████████████| 890kB 51.7MB/s 
     |████████████████████████████████| 1.1MB 45.4MB/s 
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [33]:
train_b=pd.read_csv("/content/drive/My Drive/minor/german_dataset/german_dataset.tsv",sep='\t',encoding='utf-8') #training data
train_b=train_b[train_b.task_2!='NONE']
test_b=pd.read_csv("/content/drive/My Drive/minor/german_dataset/hasoc_de_test_gold.tsv",sep='\t',encoding='utf-8') #testing data
test_b=test_b[test_b.task_2!='NONE']
x_train,x_test,y_train,y_test=train_test_split(train_b['text'],train_b['task_2'],test_size=0.15,random_state=42) #training and validation split
target_names=['OFFN','PRFN','HATE'] #names of classes

In [10]:
#convertig the tweets to list of strings and labels to list data type
x_train=list(x_train)
x_test=list(x_test)
y_train=list(y_train)
y_test=list(y_test)

In [13]:
import ktrain
from ktrain import text
MODEL_NAME = 'bert-base-german-cased' #bert predefined model
t = text.Transformer(MODEL_NAME, maxlen=500, classes=target_names) #load model, text and label preprocessor for bert
trn = t.preprocess_train(x_train, y_train) #preprocess text and labels for training data
val = t.preprocess_test(x_test, y_test) #preprocess text and labels for validation data
model = t.get_classifier() #get bert classifier
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=3) #get trainable learner for bert

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:398: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')
/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:520: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


preprocessing train...
language: de
train sequence lengths:
	mean : 26
	95percentile : 44
	99percentile : 48


Is Multi-Label? False
preprocessing test...
language: de
test sequence lengths:
	mean : 23
	95percentile : 42
	99percentile : 47


In [14]:
learner.fit_onecycle(5e-5, 4) #training for epochs=4 ,learning rate=5e-5



begin training using onecycle policy with max lr of 5e-05...
Train for 115 steps, validate for 2 steps
Epoch 1/4
115/115 [==============================] - 79s 689ms/step - loss: 1.0284 - accuracy: 0.4841 - val_loss: 1.0629 - val_accuracy: 0.4677
Epoch 2/4
115/115 [==============================] - 63s 551ms/step - loss: 0.8565 - accuracy: 0.6087 - val_loss: 1.1092 - val_accuracy: 0.5323
Epoch 3/4
115/115 [==============================] - 63s 551ms/step - loss: 0.3991 - accuracy: 0.8580 - val_loss: 1.5131 - val_accuracy: 0.4677
Epoch 4/4
115/115 [==============================] - 63s 551ms/step - loss: 0.0646 - accuracy: 0.9913 - val_loss: 1.7008 - val_accuracy: 0.5161


In [15]:
learner.validate() #predictions metrics for validation data

              precision    recall  f1-score   support

           0       0.50      0.41      0.45        17
           1       0.52      0.82      0.64        28
           2       0.50      0.12      0.19        17

    accuracy                           0.52        62
   macro avg       0.51      0.45      0.43        62
weighted avg       0.51      0.52      0.46        62



array([[ 7, 10,  0],
       [ 3, 23,  2],
       [ 4, 11,  2]])

In [16]:
predictor = ktrain.get_predictor(learner.model, preproc=t) #get predictor for the trained model of  bert

In [34]:
test_b_text=list(test_b['text']) #get text for testing
test_b['task_2']=test_b['task_2'].map({'OFFN':1,'PRFN':2,'HATE':0})
test_b_label=list(test_b['task_2']) #get label of testing

# val = t.preprocess_test(x_test, y_test) #preprocess text and labels for validation data

In [23]:
# predicted probability scores for each category
pred_y=predictor.predict_proba(test_b_text)

In [24]:
pred_y=np.argmax(pred_y,axis=1) #get class with maximum probability

In [25]:
from sklearn.metrics import classification_report

In [35]:
print(classification_report(test_b_label,pred_y)) # classification report

              precision    recall  f1-score   support

           0       0.56      0.24      0.34        41
           1       0.62      0.94      0.74        77
           2       1.00      0.06      0.11        18

    accuracy                           0.61       136
   macro avg       0.72      0.41      0.40       136
weighted avg       0.65      0.61      0.54       136

